In [ ]:
import os
from dotenv import load_dotenv

load_dotenv()

openai_api_key = os.getenv("OPENAI_API_KEY")

os.environ["OPENAI_API_KEY"] = openai_api_key


In [4]:
from openai import OpenAI

client = OpenAI()

In [2]:
from pydantic import BaseModel
from typing import Optional

In [3]:
class User(BaseModel):
    name: str
    age: int
    email: Optional[str] = None 

In [5]:
completion = client.beta.chat.completions.parse(
    model= "gpt-4o-mini",
    messages=[
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": "Make up a user."},
    ],
    response_format=User
)

In [6]:
user = completion.choices[0].message.parsed
user

User(name='Alice Johnson', age=30, email='alice.johnson@example.com')

The social media mention structure

In [7]:
from enum import Enum
from typing import List, Literal

class Mention(BaseModel):

    product: Literal['app','website','not_applicable'] # Use to choose from these
    sentiment: Literal['positive','negative','neutral']

    need_response: bool # can choose response to user
    response: Optional[str]


    support_ticket_descripton: Optional[str] # If support ticket need to open, model write description for developers

In [8]:
# Example
mentions = [
    # About the app
    "@techcorp your app is amazing! The new design is perfect",
    # Website is down, negative sentiment + needs a fix
    "@techcorp website is down again, please fix!",
    # Nothing to respond to
    "hey @techcorp you're so evil"
]

In [9]:
def analyze_mention(mention: str, personality: str="friendly")-> Mention:
    completions = client.beta.chat.completions.parse(
        model = "gpt-4o-mini",
        messages=[
            {
                "role": "system",
                "content": f"""
                    Extract structured information from 
                    social media mentions about our products.

                    Provide
                    - The product mentioned (website, app, not applicable)
                    - The mention sentiment (positive, negative, neutral)
                    - Whether to respond (true/false). Don't respond to 
                    inflammatory messages or bait.
                    - A customized response to send to the user if we need 
                    to respond.
                    - An optional support ticket description to create.

                    Your personality is {personality}.
                """
            },
            {"role": "user", "content": mention}
        ],
        response_format=Mention
    )

    return completions.choices[0].message.parsed

In [14]:
print("User post:", mentions[0])
processed_mention = analyze_mention(mentions[0])
processed_mention

User post: @techcorp your app is amazing! The new design is perfect


Mention(product='app', sentiment='positive', need_response=True, response="Thank you so much for your kind words! We're thrilled to hear you love the new design. If you have any feedback or suggestions, feel free to share!", support_ticket_descripton=None)

In [15]:
rude_mention = analyze_mention(mentions[0], personality="rude")
rude_mention.response

"Thanks for the compliment! We're glad you love the new design, but try not to get too attached. We change things around for fun sometimes."

In [16]:
mention_json_string = processed_mention.model_dump_json(indent=2)
print(mention_json_string)

{
  "product": "app",
  "sentiment": "positive",
  "need_response": true,
  "response": "Thank you so much for your kind words! We're thrilled to hear you love the new design. If you have any feedback or suggestions, feel free to share!",
  "support_ticket_descripton": null
}


Extra Practice

In [17]:
class UserPost(BaseModel):
    message: str

In [18]:
def make_post(output_class):
    completion = client.beta.chat.completions.parse(
        model="gpt-4o-mini",
        messages=[
            {
                'role': 'system',
                'content': f""" 
                    You are a customer of Tech Corp (@techcorp), a company
                    that provides an app and a website. Create a small 
                    microblog-style post to them that sends some kind of 
                    feedback, positive or negative.
                """
            },
            {'role': 'user', 'content': "Write a post please"},
        ],
        response_format=output_class
    )

    return completion.choices[0].message.parsed


new_post = make_post(UserPost)
new_post

UserPost(message='🌟 Love the new updates on the Tech Corp app! The user interface is so intuitive and smooth! 🙌 It makes managing my tasks so much easier. Keep up the great work! 👍 #TechCorp #UserFriendly')

In [19]:
analyze_mention(new_post.message)

Mention(product='app', sentiment='positive', need_response=True, response="Thank you so much for your kind words! 🌟 We're thrilled to hear that you love the new updates and find the app user-friendly. Your feedback keeps us motivated to improve! If you have any suggestions, feel free to share. 😊", support_ticket_descripton=None)

In [20]:
class UserPostWithExtras(BaseModel):
    user_mood: Literal["awful","bad","evil"]
    product: Literal['app','website','not_applicable']
    sentiment: Literal['positive','negative','neutral']
    internal_monologue: List[str]
    message: str

new_post = make_post(UserPostWithExtras)
new_post

UserPostWithExtras(user_mood='bad', product='app', sentiment='negative', internal_monologue=['Why is the app crashing so much lately?', 'I need to get my tasks done but the app keeps freezing.', 'This is so frustrating!'], message="I've been having so many issues with the app lately. It keeps crashing in the middle of my tasks, which is really frustrating. Hope you can fix these bugs soon!")

In [21]:
analyze_mention(new_post.message)

Mention(product='app', sentiment='negative', need_response=True, response="Hi there! I'm really sorry to hear you're experiencing crashes with our app. We understand how frustrating that can be. Could you provide us with more details about the device you're using or the actions leading to the crashes? This will help us resolve the issues more effectively. Thank you for your patience!", support_ticket_descripton='User is experiencing app crashes. Needs investigation and bug fixing.')

Save data into pd dataframe

In [23]:
rows = []
for mention in mentions:
    processed_mention = analyze_mention(mention)

    print(processed_mention,mention)

    processed_dict = processed_mention.model_dump()

    processed_dict['mention'] = mention
    rows.append(processed_dict)

    print("")

product='app' sentiment='positive' need_response=True response="Thank you so much for your kind words! We're thrilled to hear you love the new design of our app! If you have any suggestions or feedback, feel free to share!" support_ticket_descripton=None @techcorp your app is amazing! The new design is perfect

product='website' sentiment='negative' need_response=True response="Hi there! We're sorry to hear that you're having trouble accessing our website. Our team is currently looking into it, and we appreciate your patience! If you have any specific issues, feel free to share them with us." support_ticket_descripton='User reports that the website is down and requests immediate attention.' @techcorp website is down again, please fix!

product='not_applicable' sentiment='negative' need_response=False response=None support_ticket_descripton=None hey @techcorp you're so evil



In [24]:
import pandas as pd

df = pd.DataFrame(rows)
df

,product,sentiment,need_response,response,support_ticket_descripton,mention
0,app,positive,True,Thank you so much for your kind words! We're t...,None,@techcorp your app is amazing! The new design ...
1,website,negative,True,Hi there! We're sorry to hear that you're havi...,User reports that the website is down and requ...,"@techcorp website is down again, please fix!"
2,not_applicable,negative,False,None,None,hey @techcorp you're so evil
